In [8]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



# Monte Carlo valuation of European call options waith Numpy

## Black-Scholes-Merton option valuation 

$$C(S_t,K,t,T,r,\sigma)=S_t \cdot N(d_1)-e^{-r(T-t)} \cdot K \cdot N(d_2))$$

$$ N(d)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^d e^{-\frac{1}{2}x^2}dx$$

$$d_1=\frac{log\frac{S_t}{K}+\left(r+\frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}}$$

$$d_2=\frac{log\frac{S_t}{K}+\left(r+\frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}}$$

$S_t\quad在时点t的标的物价格水平$

$K\quad期权行权价格$

$T\quad期权到期日$

$r\quad固定无风险短期利率$

$\sigma\quad标的物固定波动率（即收益的标准差）$

$$BSM模型中欧式期权的Vega(即C对波动率\sigma的偏微分)$$

$$\frac{\partial C}{\partial\sigma}=S_tN(d_1)\sqrt{T-t}$$


$$BSM的随机微分方程（SDE)$$
$$dS_t=rS_tdt+\sigma S_tdZ_t$$
$其中Z是布朗运动$

$$SDE的欧拉离散$$
$$S_t=S_{t-\vartriangle t}exp\left(\left(r-\frac{1}{2}\sigma ^2\right)\vartriangle t + \sigma\sqrt{\vartriangle t}z_t\right)$$
$其中Z是标准正态分布随机变量$

$$欧式看涨期权的蒙特卡洛估算函数$$
$$ C_0 \approx e^{-rT}\frac{1}{I}\sum_{I}{h_T(S_T(i))}$$

## Pure python implementation

In [14]:
#pure python implementation

from time import time
from math import exp, sqrt, log
from random import gauss, seed

seed(20000)
start=time()

#Parameters.St 在时点t的标的物价格水平；K期权行权价格；T期权到期日；r 固定无风险短期利率; sigma 标的物固定波动率（即收益标准差）
S0=100.;K=105.;T=1.0;r=0.05;sigma=0.2;
M=50; dt=T/M;I=250000

#Simulating I paths with M time steps
S=[]
for i in range(I):
    path=[]
    for t in range(M+1):
        if t == 0:
            path.append(S0)
        else:
            z = gauss(0.0,1.0)
            St = path[t-1]*exp((r-0.5*sigma**2)*dt+sigma*sqrt(dt)*z)
            path.append(St)
        
    S.append(path)

#Calculating the Monte Carlo estimator
C0 = math.exp(-r * T) *sum([max(path[-1]-K,0) for path in S])/ I

#Results output
end =time()

print ("European Option Value %7.3f" %C0)
print ("Duration in Seconds   %7.3f" %(end-start))
                

European Option Value   7.999
Duration in Seconds    24.841


## Numpy Vector Implementation

In [9]:
#numpy vector implementation

import math
import numpy as np
from time import time

np.random.seed(20000)
start=time()

#parameters. St 在时点t的标的物价格水平；K期权行权价格；T期权到期日；r 固定无风险短期利率; sigma 标的物固定波动率（即收益标准差）
S0=100.;K=105.;T=1.0;r=0.05;sigma=0.2;
M=50; dt=T/M;I=250000

#simulating I paths with M time steps
S = np.zeros((M+1,I))
S[0] = S0;
for t in range(1,M+1):
    z = np.random.standard_normal(I)  #pesudorandom numbers
    S[t] = S[t-1] * np.exp((r-0.5*sigma**2)*dt + sigma * math.sqrt(dt) * z) #vectorized operation per time step over all paths

#Calculating the Monte Carlo estimator
C0 = math.exp(-r * T) * np.sum(np.maximum(S[-1]-K,0))/ I

#Result output

end = time()

print ("European Option Value %7.3f" %C0)
print ("Duration in Seconds   %7.3f" %(end-start))
    


European Option Value   8.037
Duration in Seconds     0.679
